In [48]:
import pandas as pd
import numpy as np

import datetime as dt
import pandas_market_calendars as mcal

from finquant.portfolio import build_portfolio

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import interact

import matplotlib.pyplot as plt
 
#data = pd.read_csv("https://raw.githubusercontent.com/leosmigel/analyzingalpha/master/sp500-historical-components-and-changes/sp500_history.csv",index_col=0)

In [49]:
class sectors:

    def __init__(self, start_date = np.NaN, end_date = np.NaN):
        self.VANGUARD = \
            [ \
            "VGT", \
            "VHT", \
            "VCR", \
            "VOX", \
            "VFH", \
            "VIS", \
            "VDC", \
            "VPU", \
            "VAW", \
            "VNQ", \
            "VDE" \
            ]

        self.SPDR = \
            [ \
            "XLK", \
            "XLV", \
            "XLY", \
            "XLC", \
            "XLF", \
            "XLI", \
            "XLP", \
            "XLU", \
            "XLB", \
            "XLRE", \
            "XLE" \
            ]

        self.sectors = \
            [ \
            "Information Technology",\
            "Health Care",\
            "Consumer Discretionary",\
            "Communication Services",\
            "Financials",\
            "Industrials",\
            "Consumer Staples",\
            "Utilities",\
            "Materials",\
            "Real Estate",\
            "Energy"\
            ]

    def get_Vanguard(self):
        return(self.VANGUARD)

    def get_sectors(self):
        return(self.sectors)
    
    def get_SPDR(self):
        return(self.SPDR) 
    
class businessCycles:
    
    def __init__(self, start_date = np.NaN, end_date = np.NaN):
        
        self.Expansion = \
        ['Financials', 'Information Technology'],\
        ['Communication Services'],\
        ['Consumer Staples'],\
        ['Health Care','Utilities']

        self.Slowdown = \
        ['Consumer Staples', 'Health Care'],\
        ['Industrials'],\
        ['Materials'],\
        ['Consumer Discretionary','Real Estate']    
        
        self.Recession = \
        ['Consumer Staples', 'Utilities'],\
        ['Industrials'],\
        ['Materials'],\
        ['Real Estate','Information Technology']            
        
        self.Recovery = \
        ['Consumer Discretionary', 'Real Estate'],\
        ['Materials'],\
        ['Health Care'],\
        ['Consumer Staples','Utilities']         
        
    def get_data(self):
        return([self.Expansion, self.Slowdown, self.Recession, self.Recovery])
        
def chart(data_):
    
    sum_ = pd.DataFrame(np.sum(np.array(data_),axis=0)).T
    sum_.columns = data_.columns
    print(sum_/len(data_))

    cycles = bus_cycles.get_data()

    for i in data_.index:
        print(i)
        x = data_.loc[i].sort_values(kind="quicksort", ascending=False)
        x_pos = x.index

        cc=['colors']*len(x.index)
        for n,val in enumerate(x):
            #print(n)
            if val<0:
                cc[n]='red'
            elif val>=0:
                cc[n]='blue'

        iter_ = 0
        #for c in cycles:
            #iter_ = iter_+1
            #print(c)
            #for i in c:
                #print(i)
                #for i_ in i:
                    #print(i_)
                    #print(x[i_])

                #print(x[c[i][0]])

        #plt.bar(x_pos, x,color=color_map(data_normalizer(likeability_scores)))
        plt.bar(x_pos, x,color = cc)

        #plt.xlabel("Energy Source")
        #plt.ylabel("Energy Output (GJ)")
        #plt.title("Energy output from various fuel sources")

        plt.xticks(range(0,len(x_pos)),x_pos, rotation = 75,size=10)

        plt.show()        

In [50]:
sectors_ = sectors()
print(sectors_.get_Vanguard())
print(sectors_.get_sectors())
print(sectors_.get_SPDR())

bus_cycles = businessCycles()

['VGT', 'VHT', 'VCR', 'VOX', 'VFH', 'VIS', 'VDC', 'VPU', 'VAW', 'VNQ', 'VDE']
['Information Technology', 'Health Care', 'Consumer Discretionary', 'Communication Services', 'Financials', 'Industrials', 'Consumer Staples', 'Utilities', 'Materials', 'Real Estate', 'Energy']
['XLK', 'XLV', 'XLY', 'XLC', 'XLF', 'XLI', 'XLP', 'XLU', 'XLB', 'XLRE', 'XLE']


In [51]:
stocks=sectors_.get_Vanguard()
names = sectors_.get_sectors()
stocks.append('^GSPC')
names.append('^GSPC')

In [52]:
#stocks = ['ORCL', 'TSLA', 'IBM','YELP', 'MSFT']
ls_key = 'Adj Close'

In [58]:
#filter_ = ['Y','6Mo','Q','M']
cutoff = dt.datetime.today().strftime('%Y-%m-%d')
#filter_ = ['Y','6Mo','Q','M']
filter_ = ['Y','6Mo','Q']
years_ = [2,3,5]

#filter_ = sum_.columns
def f3(Y,X):
    end_ = dt.datetime.strptime(cutoff, "%Y-%m-%d").date()

    process=True
    
    if Y=="Y":
        unit=52*1
    elif Y=="6Mo":
        unit=26*2
    elif Y=="Q":
        unit=13*4
    #elif Y=="M":
        #start_ = end_ - dt.timedelta(months=12*X)
        #process=False
        
    if process:
        start_ = end_ - dt.timedelta(weeks=unit*X)

    nyse = mcal.get_calendar('NYSE')

    #max lag due to indicators
    n = 26

    start = start_#date_time[0]
    early_start = (dt.datetime.strptime(start.strftime('%Y-%m-%d'), '%Y-%m-%d') - pd.tseries.offsets.Week(n=unit) - pd.tseries.offsets.BusinessDay(n = np.max([n])+5)).strftime('%Y-%m-%d')
    print(early_start)
    end = (dt.datetime.strptime(str(end_), '%Y-%m-%d') + pd.tseries.offsets.BusinessDay(n = 1)).strftime('%Y-%m-%d')#date_time[-1]
    print(end)

    early = nyse.schedule(start_date=early_start, end_date=end)

    #import pandas_datareader.data as web
    #f = web.DataReader(stocks, 'yahoo',early_start,end)

    pf_pre = build_portfolio(
        names=stocks, start_date=early_start, end_date=end, data_api="yfinance"
    )

    if Y=="Y":
        data_ = pf_pre.data.pct_change().asfreq('Y', method='pad').iloc[1:]
    else:
        data_ = pf_pre.data.pct_change().asfreq('Q', method='pad').iloc[1:]
    #templist = sectors_.get_sectors()
    data_.columns = names
    data_.index = data_.index.strftime('%Y-%m-%d')
    #Y = x
    #output_slider_variable.value
    #internalFilter = filter_.copy()
    #internalFilter.remove(Y)
    #all_data_ = pd.concat([data_[Y],data_[internalFilter]], axis=1)    
    #print(all_data_.describe())
    #display(all_data_.describe())
    display(data_.style.background_gradient(cmap ='RdYlGn',axis=0))
    chart(data_)
    return(data_)

out = widgets.interact(f3, Y=filter_,X=years_)
#interactive(f3, Y=filter_,X=years_)

#output_slider_variable.observe(f4, 'value')

print("choose Y")
#display(out)
#out.observe(on_value_change, names='value')    
data_ = out.widget.result
#display(data_.style.background_gradient(cmap ='RdYlGn',axis=0))

interactive(children=(Dropdown(description='Y', options=('Y', '6Mo', 'Q'), value='Y'), Dropdown(description='X…

choose Y


In [61]:
from pybloqs import Block
import pybloqs.block.table_formatters as tf
from IPython.core.display import display, HTML

dfoo =pd.DataFrame(data_)

fmt_header = tf.FmtHeader(rotate_deg=60)
table_block = Block(dfoo, formatters=[fmt_header])

display(HTML(table_block.render_html()))
table_block.save('Table.html')

2019-12-31,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.01,0.01,0.01,0.00
2020-12-31,0.00,0.01,-0.00,0.01,0.01,0.00,0.01,0.01,0.00,0.01,-0.01,0.01


'Table.html'

In [55]:
'''
dates = sp500_adjustments['date'][sp500_adjustments['date']<=cutoff].unique()

for d in dates:
    print(d)
    
    #sp500_adjustments['variable']=="added_ticker"] && 
    adds = sp500_adjustments[((sp500_adjustments['date']<=d) & (sp500_adjustments['variable']=="added_ticker"))]
    add_names, add_counts = np.unique(adds.name, return_counts=True)

    dels = sp500_adjustments[((sp500_adjustments['date']<=d) & (sp500_adjustments['variable']=="removed_ticker"))]
    del_names, del_counts = np.unique(dels.name, return_counts=True)

    df = pd.concat([pd.concat([pd.DataFrame(add_names),pd.DataFrame(add_counts)],axis=1,ignore_index=1),pd.concat([pd.DataFrame(del_names),pd.DataFrame(del_counts*-1)],axis=1,ignore_index=1)],axis=0,ignore_index=1)
    df.columns = ['name','tally']
    
    grouper = df.groupby('name')
    #res = grouper.count()
    aggregate = grouper.tally.sum()

    sp500 = pd.DataFrame(aggregate[pd.DataFrame(aggregate).tally>0]).index
    #print(sp500)
    #print(len(sp500))
    
'''

'\ndates = sp500_adjustments[\'date\'][sp500_adjustments[\'date\']<=cutoff].unique()\n\nfor d in dates:\n    print(d)\n    \n    #sp500_adjustments[\'variable\']=="added_ticker"] && \n    adds = sp500_adjustments[((sp500_adjustments[\'date\']<=d) & (sp500_adjustments[\'variable\']=="added_ticker"))]\n    add_names, add_counts = np.unique(adds.name, return_counts=True)\n\n    dels = sp500_adjustments[((sp500_adjustments[\'date\']<=d) & (sp500_adjustments[\'variable\']=="removed_ticker"))]\n    del_names, del_counts = np.unique(dels.name, return_counts=True)\n\n    df = pd.concat([pd.concat([pd.DataFrame(add_names),pd.DataFrame(add_counts)],axis=1,ignore_index=1),pd.concat([pd.DataFrame(del_names),pd.DataFrame(del_counts*-1)],axis=1,ignore_index=1)],axis=0,ignore_index=1)\n    df.columns = [\'name\',\'tally\']\n    \n    grouper = df.groupby(\'name\')\n    #res = grouper.count()\n    aggregate = grouper.tally.sum()\n\n    sp500 = pd.DataFrame(aggregate[pd.DataFrame(aggregate).tally>0]).i